In [82]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

ImportError: No module named 'geocoder'

In [83]:
import numpy as np
import pandas as pd
import urllib3
import certifi
import requests

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim

In [56]:
# Wikipedia page URL
page_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Create a PoolManager that verifies certificates when making requests. 
# See: https://urllib3.readthedocs.io/en/latest/user-guide.html#ssl
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

# Fetch the desired Wiki page
page = http.request('GET', page_url)

In [58]:
# Create a BeautifulSoup object and get the associated table 
soup = BeautifulSoup(page.data, 'lxml')
table = soup.table

In [59]:
# Use pandas read_html function to convert the table object into a DataFrame
df_import = pd.read_html(str(table))[0]
df_import.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [60]:
# Use the first row as column names. Then we drop the first row and reset the index of the entire DataFrame.
df_import.columns = df_import.iloc[0]
df_import = df_import.drop([0])

# Ignore borough that is "Not assigned" and reset DataFrame index
df_import = df_import[ df_import.Borough != 'Not assigned' ]
df_import = df_import.reset_index(drop=True)

In [61]:
df_import.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [62]:
# Get a list of index of "Not assigned" neighrourhoods 
index_list = df_import[ df_import.Neighbourhood == 'Not assigned' ].index

# Assign these neighbourhoods with corresponding Borough names.
for i in index_list:
    df_import.at[i, 'Neighbourhood'] = df_import.at[i, 'Borough']

# Double check if there are leftovers...    
#index_list = df[ df.Neighbourhood == 'Not assigned' ].index
#print(len(index_list))

In [63]:
df = df_import.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [64]:
#pd.set_option('display.max_rows', len(df))
df.head(15)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [65]:
df.shape

(103, 3)

In [69]:
df.head(15)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [72]:
toronto = df[ df['Borough'].str.contains('Toronto') ].reset_index(drop=True)
toronto

,Postcode,Borough,Neighbourhood
0,M4E,East Toronto,The Beaches
1,M4K,East Toronto,"The Danforth West, Riverdale"
2,M4L,East Toronto,"The Beaches West, India Bazaar"
3,M4M,East Toronto,Studio District
4,M4N,Central Toronto,Lawrence Park
5,M4P,Central Toronto,Davisville North
6,M4R,Central Toronto,North Toronto West
7,M4S,Central Toronto,Davisville
8,M4T,Central Toronto,"Moore Park, Summerhill East"
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi..."


In [76]:
toronto.loc[0, 'Postcode']

'M4E'

In [79]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']